<a href="https://colab.research.google.com/github/RehmaNadeem2424/AGENTIC_AI/blob/main/Assignment_NO_5_Exploring_2_0_Exp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade --quiet google-genai

In [2]:
from google.colab import userdata
import os
GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY_4')
os.environ['GEMINI_API_KEY'] = GEMINI_API_KEY

In [33]:
from IPython.display import display, Audio ,Markdown

In [34]:
from google import genai
from google.genai import Client

client: Client = genai.Client(
    api_key=GEMINI_API_KEY,
)

model: str = "gemini-2.0-flash-exp"

In [45]:
from google.colab import files

# Upload video file
uploaded = files.upload()

# List uploaded files
for filename in uploaded.keys():
    print(f'Uploaded file: {filename}')

Saving WhatsApp Video 2024-12-28 at 00.03.42_458040bb.mp4 to WhatsApp Video 2024-12-28 at 00.03.42_458040bb.mp4
Uploaded file: WhatsApp Video 2024-12-28 at 00.03.42_458040bb.mp4


In [46]:
import time

def upload_video(video_file_name):
  video_file = client.files.upload(path="my_video.mp4")
  while video_file.state == "PROCESSING":
      print('Waiting for video to be processed.')
      time.sleep(10)
      video_file = client.files.get(name=video_file.name or "")

  if video_file.state == "FAILED":
    raise ValueError(video_file.state)
  print(f'Video processing complete: ' + (video_file.uri or ""))

  return video_file

pottery_video = upload_video('pottery.mp4')

Waiting for video to be processed.
Video processing complete: https://generativelanguage.googleapis.com/v1beta/files/ru4gz1qzgcm2


In [47]:
from google.genai.types import Content, Part
prompt = """For each scene in this video,
            generate captions that describe the scene along with any spoken text placed in quotation marks.
            Place each caption into an object with the timecode of the caption in the video.
         """

video = pottery_video

response = client.models.generate_content(
    model=model,
    contents=[
        Content(
            role="user",
            parts=[
                Part.from_uri(
                    file_uri=video.uri or "",
                    mime_type=video.mime_type or ""),
                ]),
        prompt,
    ]
)

Markdown(response.text)

```json
[
  {
    "timecode": "00:00",
    "caption": "A woman with dark hair is wearing a pink and white patterned headscarf and a light pink patterned garment, speaking directly to the camera. She says, \"My name is Rahma Nadeem. I am a student at B I A I C batch 62 quarter two. I am learning about Identic AI and I have experience with Python programming.\""
  }
]
```

In [48]:
!pip install gtts --quiet

In [49]:
from gtts import gTTS
from google.genai.types import Content, Part


# Your prompt
prompt = """For each scene in this video,
            generate captions that describe the audio along with any spoken text placed in quotation marks.
            Place each caption into an object with the timecode of the caption in the video.
         """

video = pottery_video

# Generate response
response = client.models.generate_content(
    model=model,
    contents=[
        Content(
            role="user",
            parts=[
                Part.from_uri(
                    file_uri=video.uri or "",
                    mime_type=video.mime_type or ""),
            ]),
        prompt,
    ]
)

# Get the text response
output_text = response.text

# Print and speak the output
print(output_text)

```json
[
  {
    "timecode": "00:00",
    "caption": "A woman is speaking. She says, \"My name is Rahma Nadeem. I am a student at BIAIC batch 62 quarter two. I am learning about identic AI. And I have experience with Python programming.\""
  }
]
```


In [50]:
def speak_output(message):
    tts = gTTS(text=message, lang='en')
    tts.save("response.mp3")
speak_output(output_text)

In [51]:

display(Audio("response.mp3", autoplay=True))